In [ ]:

# coding: utf-8

# In[2]:


import pandas as pd
from datetime import datetime


# In[3]:


xls = pd.ExcelFile('Annotation_Sheet(P108&P102&P113 - Edit).xlsx')

dfs = []
for sheet_name in xls.sheet_names:
    df = pd.read_excel('Annotation_Sheet(P108&P102&P113 - Edit).xlsx', sheetname=sheet_name)
    dfs.append(df)
    
annotation = pd.concat(dfs, join='inner')



# In[5]:
#Adding the Start time and end time of the clips as new names with date

annotation = annotation[annotation["Date"].notnull()]
annotation = annotation[annotation["StartClock"].notnull()]
annotation = annotation[annotation["EndClock"].notnull()]
#Because need all 3 {Date,StartClock, Endclock} to be not null

annotation['DateStartTimeClock'] = pd.to_datetime(annotation['Date'].apply(str)+' '+annotation['StartClock'].apply(str))
annotation['DateEndTimeClock'] = pd.to_datetime(annotation['Date'].apply(str)+' '+annotation['EndClock'].apply(str))
print(annotation.shape)
print(annotation['DateStartTimeClock'].head())
print(annotation.dtypes)


def fetchSignal(file_, df):
    print(df.shape)
    participant = file_.split("_")[0][-3:]
    print(participant)
    #fetch the single annoataion of the particpant
    participantannotation = annotation.loc[annotation['ParticipantID'] == float(participant)]
    print(participantannotation.shape)
    print(annotation.shape)
    tempframe = pd.DataFrame()
    # all the rows in the annotation
    for index, row in participantannotation.iterrows():
        signalrange = (df['Timestamp'] >= row['DateStartTimeClock']) & (df['Timestamp'] < row['DateEndTimeClock'])
        signalframe = df.loc[signalrange]
        signalframe['activity1'] = row['ActivityLevel1']
        signalframe['activity2'] = row['ActivityLevel2']
        signalframe['activity3'] = row['ActivityLevel3']
        signalframe['Accel_Waist'] = row['Accel_Waist']
        signalframe['Accel_Wrist'] = row['Accel_Wrist']
        signalframe['Sequence'] = row['Sequence']
        signalframe['ParticipantID'] = row['ParticipantID']
        tempframe = tempframe.append(signalframe)
    df = tempframe
    print(df.shape)
    print(df.head)
    return df


# In[ ]:


#Read all csv files in the folder and ignore the first 10 lines(The first 10 lines are explanation that is not neccesarry)
import glob
import pandas as pd
allFiles = glob.glob("../../ActivityRecognitionDataCSV/*waist*.csv")
list_ = []
#print(allFiles)
#file = 101.RAW.CSV
#df = df of 101 data  of gt3x
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,skiprows=10) #skiprows = ignore 10 first lines
    print("Processing file ", file_)
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) #convert string into datetime format
    df3 = fetchSignal(file_,df)  ##??
    list_.append(df3)
    print("Finished Processing file ", file_)
dataframe = pd.concat(list_)

#print("Column headings:")
#print(dataframe.columns)
#print(dataframe.dtypes)
#print(dataframe.shape)
print(dataframe.head())


# ### Save the big cleaned dataframe in the pickle

# In[ ]:




# In[ ]:



(4294, 17)
0   2018-01-18 16:07:40
1   2018-01-18 16:11:11
2   2018-01-18 16:11:17
3   2018-01-18 16:11:20
4   2018-01-18 16:11:23
Name: DateStartTimeClock, dtype: datetime64[ns]
ParticipantID                         float64
Accel_Waist                            object
Accel_Wrist                            object
Date                           datetime64[ns]
Sequence                              float64
VideoName                              object
VideoStartClock_camera                 object
VideoStartClock_timeanddate            object
StartRunTime                           object
EndRunTime                             object
StartClock                             object
EndClock                               object
ActivityLevel1                        float64
ActivityLevel2                        float64
ActivityLevel3                        float64
DateStartTimeClock             datetime64[ns]
DateEndTimeClock               datetime64[ns]
dtype: object
Processing file  ../../Ac

In [ ]:
import pickle
# ?? how to activate pickle
pickling_on = open("wholedatasignal.pickle","wb")
pickle.dump(dataframe, pickling_on)


# In[ ]:


dataframe = dataframe.dropna() #drops rows or columns that have missing value
dataframe = dataframe.reset_index(drop=True)


# In[ ]:


dataframe.shape

In [ ]:



import itertools
import numpy as np
from io import StringIO
import math
import scipy.stats.stats as stats
import matplotlib.pyplot as plt 
import pandas as pd


# ### Splitting two second window for activity level 3
#def differentclipsizeact3(x,picklename):
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 300) and ((actvalue == data.iloc[ind,6]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        actvalue = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 300):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,6]
    subvalue = data.iloc[ind,10]

dfal3 = pd.DataFrame(d)
dfal3.head()

#differentclipsizeact3(30,"dfal31sec.pickle")
#differentclipsizeact3(60,"dfal32sec.pickle")
#differentclipsizeact3(150,"dfal35sec.pickle")
#differentclipsizeact3(300,"dfal310sec.pickle")
#differentclipsizeact3(450,"dfal315sec.pickle")
#differentclipsizeact3(600,"dfal320sec.pickle")
#differentclipsizeact3(750,"dfal325sec.pickle")





In [ ]:
pickling_on = open("dfal10sec.pickle","wb") #activity 3 
pickle.dump(dfal3, pickling_on)
pickling_on.close()


In [ ]:




# ### Splitting two second window for activity level 3
#def differentclipsizeact3(x,picklename):
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 450) and ((actvalue == data.iloc[ind,6]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        actvalue = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 450):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,6]
    subvalue = data.iloc[ind,10]

dfal31 = pd.DataFrame(d)
dfal31.head()


In [ ]:
pickling_on1 = open("dfaltest15sec.pickle","wb") #activity 3 
pickle.dump(dfal31, pickling_on1)
pickling_on1.close()

In [ ]:
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 600) and ((actvalue == data.iloc[ind,6]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        actvalue = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 600):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,6]
    subvalue = data.iloc[ind,10]

dfal32 = pd.DataFrame(d)
dfal32.head()

In [ ]:
pickling_on2 = open("dfaltest20sec.pickle","wb") #activity 3 
pickle.dump(dfal32, pickling_on2)
pickling_on2.close()

In [ ]:
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 60) and ((actvalue == data.iloc[ind,6]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        actvalue = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 60):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,6]
    subvalue = data.iloc[ind,10]

dfal33 = pd.DataFrame(d)
dfal33.head()

In [ ]:
pickling_on3 = open("dfaltest2sec.pickle","wb") #activity 3 
pickle.dump(dfal33, pickling_on3)
pickling_on3.close()

In [ ]:
data = None
data = dataframe
x = 0
i = 0
j = 0
array = []
d = []
actvalue = ''
subvalue = ''
for ind, ro in data.iterrows():
    if ((j < 30) and ((actvalue == data.iloc[ind,6]) or actvalue == '') and ((subvalue == data.iloc[ind,10]) or subvalue == '')) :
        activity1 = data.iloc[ind,4]
        activity2 = data.iloc[ind,5]
        actvalue = data.iloc[ind,6]
        waist = data.iloc[ind,7]
        wrist = data.iloc[ind,8]
        sequence = data.iloc[ind,9]
        subvalue = data.iloc[ind,10]
        array.append([data.iloc[ind,1],data.iloc[ind,2],data.iloc[ind,3]])
        j+=1
    else :
        if(j == 30):
            d.append([subvalue,actvalue,array])
        array = []
        i+=1
        j = 0
    actvalue = data.iloc[ind,6]
    subvalue = data.iloc[ind,10]

dfal34 = pd.DataFrame(d)
dfal34.head()

In [ ]:
pickling_on4 = open("dfaltest1sec.pickle","wb") #activity 3 
pickle.dump(dfal34, pickling_on4)
pickling_on4.close()